In [12]:
import datetime
import pandas as pd
import numpy as np
import pymysql

In [13]:
# 공휴일 데이터 불러오기
df_holiday = pd.read_csv("D:/WORKSPACE/project/1st_miniproject/Mook/hd_21_24.csv")

In [14]:
# 데이터 확인
print(df_holiday.head(10))
print(df_holiday.info())

           일시 dateName
0  2021-01-01     1월1일
1  2021-02-11       설날
2  2021-02-12       설날
3  2021-02-13       설날
4  2021-03-01      삼일절
5  2021-05-05     어린이날
6  2021-05-19   부처님오신날
7  2021-06-06      현충일
8  2021-08-15      광복절
9  2021-09-21       추석
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   일시        70 non-null     object
 1   dateName  70 non-null     object
dtypes: object(2)
memory usage: 1.2+ KB
None


In [15]:
# 공휴일 데이터의 일시를 날짜 데이터로 변경
df_holiday['일시'] = pd.to_datetime(df_holiday['일시'])

In [16]:
# 날짜 데이터 생성(2022-01-01 ~ 2024-10-31 )
date_range = pd.date_range(start='2022-01-01', end='2024-10-31', freq='D')

In [17]:
# 데이터프레임으로 저장
df_date = pd.DataFrame(date_range, columns=['date'])

In [18]:
# 날짜 데이터에 공휴일이거나 주말이면 1 아니면 0이라는 컬럼 추가
df_date['holiday_or_weekend'] = df_date['date'].apply(
    lambda x: 1 if (x.dayofweek >= 5 or x in df_holiday['일시'].values) else 0
)

In [19]:
# 확인
print(df_date.head(10))
print(df_date.info())

        date  holiday_or_weekend
0 2022-01-01                   1
1 2022-01-02                   1
2 2022-01-03                   0
3 2022-01-04                   0
4 2022-01-05                   0
5 2022-01-06                   0
6 2022-01-07                   0
7 2022-01-08                   1
8 2022-01-09                   1
9 2022-01-10                   0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   date                1035 non-null   datetime64[ns]
 1   holiday_or_weekend  1035 non-null   int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 16.3 KB
None


In [20]:
# db 연결
conn = pymysql.connect(host="localhost", port=3306, db="project", 
                       user="humanda", password="humanda")

In [21]:
# 명령 도구 준비
cursor = conn.cursor()

In [22]:
# 명령 실행 (테이블 생성)
sql = """create table dates
         (
            ID INT AUTO_INCREMENT PRIMARY KEY,         
            date date,
            holiday_or_weekend int
         )"""
cursor.execute(sql)

0

In [23]:
# 명령 실행 (데이터 삽입)
sql2 = "insert into dates(date, holiday_or_weekend) values (%s, %s)"
for index, row in df_date.iterrows():
    cursor.execute(sql2, tuple(row))

In [24]:
# 원본 테이블에 적용
conn.commit()

In [25]:
# 명령 실행 (조회)
sql3 = "select * from dates"
cursor.execute(sql3)

1035

In [26]:
# 조회 결과 읽기
data = cursor.fetchall()

In [27]:
# 표시
for row in data:
    print(row)

(1, datetime.date(2022, 1, 1), 1)
(2, datetime.date(2022, 1, 2), 1)
(3, datetime.date(2022, 1, 3), 0)
(4, datetime.date(2022, 1, 4), 0)
(5, datetime.date(2022, 1, 5), 0)
(6, datetime.date(2022, 1, 6), 0)
(7, datetime.date(2022, 1, 7), 0)
(8, datetime.date(2022, 1, 8), 1)
(9, datetime.date(2022, 1, 9), 1)
(10, datetime.date(2022, 1, 10), 0)
(11, datetime.date(2022, 1, 11), 0)
(12, datetime.date(2022, 1, 12), 0)
(13, datetime.date(2022, 1, 13), 0)
(14, datetime.date(2022, 1, 14), 0)
(15, datetime.date(2022, 1, 15), 1)
(16, datetime.date(2022, 1, 16), 1)
(17, datetime.date(2022, 1, 17), 0)
(18, datetime.date(2022, 1, 18), 0)
(19, datetime.date(2022, 1, 19), 0)
(20, datetime.date(2022, 1, 20), 0)
(21, datetime.date(2022, 1, 21), 0)
(22, datetime.date(2022, 1, 22), 1)
(23, datetime.date(2022, 1, 23), 1)
(24, datetime.date(2022, 1, 24), 0)
(25, datetime.date(2022, 1, 25), 0)
(26, datetime.date(2022, 1, 26), 0)
(27, datetime.date(2022, 1, 27), 0)
(28, datetime.date(2022, 1, 28), 0)
(29, datet

In [28]:
# db에서 불러와 데이터프레임으로 저장
df_date = pd.read_sql_query(sql3, conn)

C:\Users\human\AppData\Local\Temp\ipykernel_26124\1084959558.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_date = pd.read_sql_query(sql3, conn)


In [29]:
df_date

,ID,date,holiday_or_weekend
0,1,2022-01-01,1
1,2,2022-01-02,1
2,3,2022-01-03,0
3,4,2022-01-04,0
4,5,2022-01-05,0
...,...,...,...
1030,1031,2024-10-27,1
1031,1032,2024-10-28,0
1032,1033,2024-10-29,0
1033,1034,2024-10-30,0


In [30]:
cursor.close()
conn.close()

In [31]:
df_date.to_csv('df_date.csv', index=False)